In [24]:
import pandas as pd
import geopandas as gpd
import urllib
import json
import pycountry

ModuleNotFoundError: No module named 'pycountry'

In [23]:
# JSON time-series of coronavirus cases (confirmed, deaths and recovered) per country - updated daily on github
with urllib.request.urlopen("https://pomber.github.io/covid19/timeseries.json") as url:
    data = json.loads(url.read().decode())

# print(data)
# creates a data frame for the confirmed cases per country and on what day, confirmed cases and recoveries are available
# deaths are the most important indicator as testing is not widely available
columns = ['entity','code','dates', 'confirmed cases']
# data = [[data[x][d]['confirmed'] for x in data.keys()] for d in range(len(data['Afghanistan']))]
gdata = [[c, data[c][d]['date'], data[c][d]['confirmed']] for c in data.keys() for d in range(len(data['Afghanistan']))]
print(gdata)
# df = pd.DataFrame(data, columns=columns)


[['Afghanistan', '2020-1-22', 0], ['Afghanistan', '2020-1-23', 0], ['Afghanistan', '2020-1-24', 0], ['Afghanistan', '2020-1-25', 0], ['Afghanistan', '2020-1-26', 0], ['Afghanistan', '2020-1-27', 0], ['Afghanistan', '2020-1-28', 0], ['Afghanistan', '2020-1-29', 0], ['Afghanistan', '2020-1-30', 0], ['Afghanistan', '2020-1-31', 0], ['Afghanistan', '2020-2-1', 0], ['Afghanistan', '2020-2-2', 0], ['Afghanistan', '2020-2-3', 0], ['Afghanistan', '2020-2-4', 0], ['Afghanistan', '2020-2-5', 0], ['Afghanistan', '2020-2-6', 0], ['Afghanistan', '2020-2-7', 0], ['Afghanistan', '2020-2-8', 0], ['Afghanistan', '2020-2-9', 0], ['Afghanistan', '2020-2-10', 0], ['Afghanistan', '2020-2-11', 0], ['Afghanistan', '2020-2-12', 0], ['Afghanistan', '2020-2-13', 0], ['Afghanistan', '2020-2-14', 0], ['Afghanistan', '2020-2-15', 0], ['Afghanistan', '2020-2-16', 0], ['Afghanistan', '2020-2-17', 0], ['Afghanistan', '2020-2-18', 0], ['Afghanistan', '2020-2-19', 0], ['Afghanistan', '2020-2-20', 0], ['Afghanistan', '2

In [11]:
shapefile = '../data/countries_110m/ne_110m_admin_0_countries.shp'

gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()


,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [12]:
print(gdf[gdf['country'] == 'Antarctica'])
gdf = gdf.drop(gdf.index[159])

        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  
